In [1]:
import pyspark.sql.functions as fn

# Load from Parquet

In [2]:
df = spark.read.parquet('dataset/THB_cards.parquet')

In [3]:
df.createOrReplaceTempView("cards")

In [4]:
df.columns

['number',
 'rarity',
 'name',
 'convertedManaCost',
 'encoded_colorIdentity',
 'colorIdentity',
 'originalText',
 'filteredText',
 'power',
 'toughness']

## Groups 

In [5]:
df.groupby('colorIdentity').count().show()

+-------------+-----+
|colorIdentity|count|
+-------------+-----+
|          G,W|    4|
|            B|   45|
|          R,W|    2|
|          G,R|    4|
|            U|   45|
|          B,W|    3|
|          B,R|    4|
|          G,U|    3|
|            W|   48|
|            R|   44|
|          B,U|    6|
|            G|   44|
|          U,W|    3|
|             |   14|
|          B,G|    2|
|          R,U|    2|
+-------------+-----+



## SQL / FN

In [6]:
df.agg(*[fn.max(c).alias('max ' + c) for c in ['power', 'toughness']]).show()

+---------+-------------+
|max power|max toughness|
+---------+-------------+
|       10|            7|
+---------+-------------+



In [7]:
spark.sql("SELECT * FROM cards WHERE power == 10").show()

+------+------+------------+-----------------+---------------------+-------------+--------------------+--------------------+-----+---------+
|number|rarity|        name|convertedManaCost|encoded_colorIdentity|colorIdentity|        originalText|        filteredText|power|toughness|
+------+------+------------+-----------------+---------------------+-------------+--------------------+--------------------+-----+---------+
|   181|  rare|Nessian Boar|                5|                    6|            G|All creatures abl...|All creatures abl...|   10|        6|
+------+------+------------+-----------------+---------------------+-------------+--------------------+--------------------+-----+---------+



In [12]:
spark.sql("""
    SELECT * FROM cards
    ORDER BY power DESC, toughness DESC
    LIMIT 5
""").show()

+------+------+--------------------+-----------------+---------------------+-------------+--------------------+--------------------+-----+---------+
|number|rarity|                name|convertedManaCost|encoded_colorIdentity|colorIdentity|        originalText|        filteredText|power|toughness|
+------+------+--------------------+-----------------+---------------------+-------------+--------------------+--------------------+-----+---------+
|   181|  rare|        Nessian Boar|                5|                    6|            G|All creatures abl...|All creatures abl...|   10|        6|
|   297|  rare|  Treeshaker Chimera|                7|                    6|            G|All creatures abl...|All creatures abl...|    8|        5|
|   292|  rare|   Demon of Loathing|                7|                    1|            B|Flying, trample
W...|FLYING, trample
W...|    7|        7|
|   150|mythic|Purphoros, Bronze...|                5|                   10|            R|Indestructible
A